# SQLite usage of SQLDatabaseChain

In [ ]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [ ]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = OpenAI(temperature=0)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("Name one album by Queen")

### Prompt customization

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the "Musicanti", what they mean is artists or bands.
Also always report a single returned row.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)


In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)

In [ ]:
db_chain.run("I need some albums by the musicanti called metallica")

### Return intermediate steps

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, return_intermediate_steps=True)

In [ ]:
result = db_chain("I need some albums by the musicanti called metallica")
result["intermediate_steps"]

### Setting 'limit' programmatically (as opposed to in custom prompt)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, top_k=3)

In [ ]:
db_chain.run("What are some example tracks by composer Beethoven?")

# THIS ERRORS WITH THE MISSING SEMICOLON AFTER 'LIMIT n'

#### Intermezzo: trying to nudge the LLM into adding a semicolon

In [ ]:
# my test
tin = db.table_info
# inp = "What are some example tracks by composer Beethoven?"
inp = "What are some example tracks by composer Sibelius?"
tpk = 5

In [ ]:
# my test
from langchain.chains.sql_database.prompt import _sqlite_prompt

my_sqlite_prompt = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Remember that SQLite statement must end with a semicolon (;).

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Question: {input}"""

In [ ]:
my_sp = my_sqlite_prompt.format(
    input=inp,
    table_info=tin,
    top_k=tpk,
)

sp = _sqlite_prompt.format(
    input=inp,
    table_info=tin,
    top_k=tpk,
)

In [ ]:
base_result = llm(sp)
my_result = llm(my_sp)

In [ ]:
print(base_result)
print(my_result)

#### Test: explicit specification of the prompt to use

In [ ]:
# vanilla sqlite still not working
from langchain.chains.sql_database.prompt import SQLITE_PROMPT
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=SQLITE_PROMPT, verbose=True)
db_chain.run("What are some example tracks by composer Beethoven?")

# THIS ERRORS WITH THE MISSING SEMICOLON AFTER 'LIMIT n'

In [ ]:
from langchain.chains.sql_database.prompt import PromptTemplate

MY_SQLITE_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=my_sqlite_prompt,
)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=MY_SQLITE_PROMPT, verbose=True, top_k=2)
db_chain.run("What are some example tracks by composer Johann Sebastian Bach?")

#### Adding sample rows in table_info for more informed results

In [ ]:
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Track'], # we include only one table to save tokens in the prompt :)
    sample_rows_in_table_info=2)

In [ ]:
print(db.table_info)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("What are some example tracks by Bach?")

#### Custom table info

In [ ]:
custom_table_info = {
    "Track": """CREATE TABLE Track (
    "TrackId" INTEGER NOT NULL, 
    "Name" NVARCHAR(200) NOT NULL,
    "Composer" NVARCHAR(220),
    PRIMARY KEY ("TrackId")
)
/*
3 rows from Track table:
TrackId Name    Composer
1   For Those About To Rock (We Salute You) Angus Young, Malcolm Young, Brian Johnson
2   Balls to the Wall   None
3   My favorite song ever   The coolest composer of all time
*/"""
}

In [ ]:
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Track', 'Playlist'],
    sample_rows_in_table_info=2,
    custom_table_info=custom_table_info)

print(db.table_info)    

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
db_chain.run("What are some example tracks by Bach?")

### SQLDatabaseSequentialChain

In [ ]:
from langchain.chains import SQLDatabaseSequentialChain
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [ ]:
chain.run("How many employees are also customers?")